<a href="https://colab.research.google.com/github/valmirf/mineracao_textual/blob/main/CNN/05b_Sentiment_Analysis_Imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Análise de Sentimentos
A análise de sentimento visa determinar a opinião ou sentimento das pessoas eem relçao a um produto, pessoa, evento, ou qualquer outro objeto de interesse. Por exemplo, um palestrante ou escritor em relação a um documento, interação ou evento. O sentimento é principalmente categorizado em categorias positivas, negativas e neutras. Por meio da análise de sentimento, podemos prever, por exemplo, a opinião e a atitude de um cliente sobre um produto com base em uma resenha que ele escreveu. Essa técnica é amplamente aplicada a coisas como revisões, pesquisas, documentos e muito mais.

##Base de Dados IMDB
O conjunto de dados de classificação de sentimento do IMDB (https://www.imdb.com/) consiste em 50000 resenhas de filmes de usuários do IMDB que são rotuladas como positivas (1) ou negativas (0). As 50000 resenhas são divididas em 25000 para treinamento e 25000 para teste. O conjunto de dados foi criado por pesquisadores da Universidade de Stanford e publicado em um artigo de 2011, onde alcançou 88.89% de precisão. 

In [ ]:
from keras.datasets import imdb

# Load the data, keeping only 10,000 of the most frequently occuring words
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)

In [ ]:
import numpy as np

# Since we restricted ourselves to the top 10000 frequent words, no word index should exceed 10000
# we'll verify this below
# Here is a list of maximum indexes in every review --- we search the maximum index in this list of max indexes
print(type([max(sequence) for sequence in train_data]))

# Find the maximum of all max indexes
max([max(sequence) for sequence in train_data])

data = np.concatenate((train_data, test_data), axis=0)
labels = np.concatenate((train_labels, test_labels), axis=0)

print("Categories:", np.unique(labels))
print("Number of unique words:", len(np.unique(np.hstack(data))))

length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Você pode ver acima que o conjunto de dados é classificado em duas categorias, 0 ou 1, que representa o sentimento da resenha, negativo e positivo respectivamente. Todo o conjunto de dados contém 9.998 palavras únicas e o comprimento médio da revisão é de 234 palavras, com um desvio padrão de 173 palavras. 

##Decodificação da Resenha
Abaixo, podemos ver a primeira revisão do conjunto de dados, que é rotulada como positiva (1). O primeiro passo é mapear as palavras em índices inteiros. 


In [ ]:
# Let's quickly decode a review

# step 1: load the dictionary mappings from word to integer index
word_index = imdb.get_word_index()

# step 2: reverse word index to map integer indexes to their respective words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Step 3: decode the review, mapping integer indices to words
#
# indices are off by 3 because 0, 1, and 2 are reserverd indices for "padding", "Start of sequence" and "unknown"
decoded_review = ' '.join([reverse_word_index.get(i-3, '?') for i in train_data[0]])

decoded_review

##Preparação dos Dados

O primeiro passo é converter os índices em Tensores para usar na biblioteca do TensorFlow. O vetor de 10000 dimensões correspondente a cada revisão terá um índice correspondente a uma palavra. Todo índice com valor 1 é uma palavra que está presente na revisão e cada índice contendo 0 é uma palavra que não está presente na revisão. Isso resultará em um tensor de tamanho (25000, 10000).

In [ ]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))    # Creates an all zero matrix of shape (len(sequences),10K)
    for i,sequence in enumerate(sequences):
        results[i,sequence] = 1                        # Sets specific indices of results[i] to 1s
    return results

# Vectorize training Data
X_train = vectorize_sequences(train_data)

# Vectorize testing Data
X_test = vectorize_sequences(test_data)

In [ ]:
X_train.shape

In [ ]:
y_train = np.asarray(train_labels).astype('float32')
y_test  = np.asarray(test_labels).astype('float32')

##Construindo a Rede Neural

Agora podemos construir uma rede neural simples. Começaremos definindo o tipo de modelo que queremos construir. Existem dois tipos de modelos disponíveis no Keras: o modelo sequencial e a classe do modelo usado na API funcional.

Em seguida, simplesmente adicionamos as camadas de entrada, ocultas e de saída. Entre eles, estamos usando o *dropout* para evitar o sobreajuste. 

Usaremos a camada "*dense*", o que significa que as unidades estão totalmente conectadas. Dentro das camadas ocultas, usamos a função relu porque é sempre um bom começo e produz um resultado satisfatório na maioria das vezes. Sinta-se à vontade para experimentar outras funções de ativação.

Na camada de saída, usamos a função sigmóide, que mapeia os valores entre 0 e 1. Observe que definimos a forma de entrada para 10000 na camada de entrada porque nossas análises têm 10000 inteiros de comprimento. A camada de entrada recebe 10000 como entrada e a produz com um formato de 50.

Por último, vemos um resumo do modelo que acabamos de construir.

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

print(model)




##Compilação do Modelo

Nessa parte do código, definimos os algoritmos de otimização, a função de perda e a métrica que será utilizada para avaliação do modelo. 

In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss = losses.binary_crossentropy,
              metrics = [metrics.binary_accuracy])

##Configuração da Avaliação
Reservaremos uma parte de nossos dados de treinamento para validação da precisão do modelo durante o treinamento. Um conjunto de validação nos permite monitorar o progresso de nosso modelo em dados não vistos anteriormente à medida que ele passa por épocas durante o treinamento.
As etapas de validação nos ajudam a ajustar os parâmetros de treinamento da função `model.fit` para evitar overfitting e underfitting de dados.


In [ ]:
# Input for Validation
X_val = X_train[:10000]
partial_X_train = X_train[10000:]

# Labels for validation
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

##Treinamento do Modelo

In [ ]:
history = model.fit(partial_X_train,
                   partial_y_train,
                   epochs=20,
                   batch_size=512,
                   validation_data=(X_val, y_val))

##Visualizando os Resultados

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

history_dict = history.history
history_dict.keys()

# Plotting losses
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values) + 1)

plt.plot(epochs, loss_values, 'bo', label="Training Loss")
plt.plot(epochs, val_loss_values, 'b', label="Validation Loss")

plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss Value')
plt.legend()

plt.show()



### Retreinando o Modelo

In [ ]:
model.fit(partial_X_train,
                   partial_y_train,
                   epochs=3,
                   batch_size=512,
                   validation_data=(X_val, y_val))

In [ ]:
# Making Predictions for testing data
np.set_printoptions(suppress=True)
result = model.predict(X_test)

result

In [ ]:
y_pred = np.zeros(len(result))
for i, score in enumerate(result):
    y_pred[i] = 1 if score > 0.5 else 0

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_pred, y_test)
mae